In [22]:
string name = "Michael";

name

Michael

# Identity and Stuff

Hi there!

In [23]:
record Person(string name, DateTime dateOfBirth) { }

var michael = new Person("Michael Perry", new DateTime(1971, 5, 10));

michael

name,Michael Perry
dateOfBirth,1971-05-10 00:00:00Z


In [24]:
int AgeOfPerson(Person person)
{
  var today = DateTime.Today;
  var age = today.Year - person.dateOfBirth.Year;
  if (person.dateOfBirth.Date > today.AddYears(-age)) age--;
  return age;
}

AgeOfPerson(michael)

52

In [25]:
using System.Collections.Immutable;

var letters = ImmutableArray.Create('a', 'b', 'c');

letters

[ a, b, c ]

In [26]:
var newArray = letters.Add('d');
var moreLetters = ImmutableArray<char>.Empty.Add('e').Add('f').Add('g');

moreLetters

[ e, f, g ]

In [27]:
newArray

[ a, b, c, d ]

In [28]:
#r "./TicTacToe/bin/Debug/net7.0/TicTacToe.dll"

using TicTacToe;

var game = Game.Empty
  .Play(4);

game.Html

<!DOCTYPE html>
 
 
 Tic-Tac-Toe Board 
 
 
 
 X

In [29]:
var moves = game.EmptySquares.Select(index =>
  game.Play(index).Html
);

moves

index,value
0,<!DOCTYPE html> Tic-Tac-Toe Board OX
1,<!DOCTYPE html> Tic-Tac-Toe Board OX
2,<!DOCTYPE html> Tic-Tac-Toe Board OX
3,<!DOCTYPE html> Tic-Tac-Toe Board OX
4,<!DOCTYPE html> Tic-Tac-Toe Board XO
5,<!DOCTYPE html> Tic-Tac-Toe Board XO
6,<!DOCTYPE html> Tic-Tac-Toe Board XO
7,<!DOCTYPE html> Tic-Tac-Toe Board XO


In [30]:
game = game.Play(1);

game.Html

<!DOCTYPE html>
 
 
 Tic-Tac-Toe Board 
 
 
 
 O X

In [32]:
Symbol Evaluate(Game game)
{
  var winner = game.Winner;
  if (winner != Symbol.Empty)
    return winner;
  if (!game.EmptySquares.Any())
    return Symbol.Empty;

  var outcomes = game.EmptySquares.Select(index =>
    Evaluate(game.Play(index))
  ).ToImmutableArray();
  if (outcomes.Any(outcome => outcome == game.NextPlayer))
    return game.NextPlayer;
  if (outcomes.Any(outcome => outcome == Symbol.Empty))
    return Symbol.Empty;
  return game.NextPlayer == Symbol.X ? Symbol.O : Symbol.X;
}

Evaluate(game)

X

In [33]:
game.EmptySquares.Select(index =>
  game.Play(index)
).Select(game =>
  game.HtmlWithOutcome(Evaluate(game))
)

index,value
0,<!DOCTYPE html> Tic-Tac-Toe Board XOXX wins
1,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
2,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
3,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
4,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
5,<!DOCTYPE html> Tic-Tac-Toe Board OXXDraw
6,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins


In [37]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"


Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.23313.2

Loading extensions from `C:\Users\micha\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.23313.2\interactive-extensions\dotnet\Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [39]:
#!connect mssql --kernel-name school "Persist Security Info=False; Integrated Security=true; Initial Catalog=School; Server=localhost; TrustServerCertificate=True;"


Kernel added: #!sql-school

In [50]:
#!sql-school

DROP TABLE IF EXISTS Course;

CREATE TABLE Course (
  CourseID int IDENTITY(1,1) NOT NULL,
  Code nvarchar(10) NOT NULL,
  Title nvarchar(50) NOT NULL,
  Credits int NOT NULL,
  CONSTRAINT PK_Course PRIMARY KEY (CourseID)
);

Commands completed successfully.

In [57]:
#!sql-school

TRUNCATE TABLE Course;
CREATE UNIQUE INDEX IX_Course_Code ON Course (Code);

Commands completed successfully.

In [69]:
#!sql-school

SELECT * FROM Course

(1 row affected)

CourseID,Code,Title,Credits
1,CHEM-101,Chemistry,3


In [66]:
#!sql-school

INSERT INTO Course (Title, Code, Credits)
SELECT 'Chemistry', 'CHEM-101', 3
WHERE NOT EXISTS (SELECT * FROM Course WHERE Code = 'CHEM-101')

(0 rows affected)

In [71]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Graphviz"

using Jinaga;
using Jinaga.Graphviz;

Installed Packages Jinaga, 0.4.0 Jinaga.Graphviz, 0.4.0

In [80]:
[FactType("School.Course")]
public record Course(string code) {}

[FactType("School.Course.Title")]
public record CourseTitle(Course course, string title, CourseTitle[] prior) {}

[FactType("School.Course.Credits")]
public record CourseCredits(Course course, int credits, CourseCredits[] prior) {}

Renderer.RenderTypes(typeof(Course), typeof(CourseTitle), typeof(CourseCredits))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 8.0.5 (20230430.1635)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- School.Course.Title -->
 
 School.Course.Title 
 
 School.Course.Title 
 
<!-- School.Course.Title->School.Course.Title -->
 
 School.Course.Title->School.Course.Title 
 
 
 prior 
 
<!-- School.Course -->
 
 School.Course 
 
 School.Course 
 
<!-- School.Course.Title->School.Course -->
 
 School.Course.Title->School.Course 
 
 
 course 
 
<!-- School.Course.Credits -->
 
 School.Course.Credits 
 
 School.Course.Credits 
 
<!-- School.Course.Credits->School.Course -->
 
 School.Course.Credits->School.Course 
 
 
 course 
 
<!-- School.Course.Credits->School.Course.Credits -->
 
 School.Course.Credits->School.Course.Credits 
 
 
 prior

In [87]:
var j = JinagaClient.Create();

var trigonometry = await j.Fact(new Course("MATH-210"));
var trigonometryTitle = await j.Fact(new CourseTitle(trigonometry, "Trigonometry", new CourseTitle[] { }));
var trigTitle = await j.Fact(new CourseTitle(trigonometry, "Trig", new CourseTitle[] { trigonometryTitle }));
var otherTitle = await j.Fact(new CourseTitle(trigonometry, "Other", new CourseTitle[] { trigonometryTitle }));

Renderer.RenderFacts(trigonometry, trigonometryTitle, trigTitle, otherTitle)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 8.0.5 (20230430.1635)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== -->
 
 5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== 
 
 School.Course 
 
 code 
 
 MATH-210 
 
 
<!-- 1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg== -->
 
 1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg== 
 
 School.Course.Title 
 
 title 
 
 Trigonometry 
 
 
<!-- 1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg==->5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== -->
 
 1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg==->5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== 
 
 
 course 
 
<!-- a+CAF9wRyccI/puBsRtKtATH9ThMsI4zO660mLMQxfL6scfAb4IJ21KP0kWOkWTCGmCcRAzOCAlfq8P+qqC0mg== -->
 
 a+CAF9wRyccI/puBsRtKtATH9ThMsI4zO660mLMQxfL6scfAb4IJ21KP0kWOkWTCGmCcRAzOCAlfq8P+qqC0mg== 
 
 School.Course.Title 
 
 title 
 
 Trig 
 
 
<!-- a+CAF9wRyccI/puBsRtKtATH9ThMsI4zO660mLMQxfL6scfAb4IJ21KP0kWOkWTCGmCcRAzOCAlfq8P+qqC0mg==->5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== -->
 
 a+CAF9wRyccI/puBsRtKtATH9ThMsI4zO660mLMQxfL6scfAb4IJ21KP0kWOkWTCGmCcRAzOCAlfq8P+qqC0mg==->5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== 
 
 
 course 
 
<!-- a+CAF9wRyccI/puBsRtKtATH9ThMsI4zO660mLMQxfL6scfAb4IJ21KP0kWOkWTCGmCcRAzOCAlfq8P+qqC0mg==->1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg== -->
 
 a+CAF9wRyccI/puBsRtKtATH9ThMsI4zO660mLMQxfL6scfAb4IJ21KP0kWOkWTCGmCcRAzOCAlfq8P+qqC0mg==->1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg== 
 
 
 prior 
 
<!-- sRevcBWen3xKQww4RWFjvUpuSEERV55MSmTjjDUiPYatzmb2FfQE5VS39VXfaZDpRDmWPZ0LSVESymUeyJQLEQ== -->
 
 sRevcBWen3xKQww4RWFjvUpuSEERV55MSmTjjDUiPYatzmb2FfQE5VS39VXfaZDpRDmWPZ0LSVESymUeyJQLEQ== 
 
 School.Course.Title 
 
 title 
 
 Other 
 
 
<!-- sRevcBWen3xKQww4RWFjvUpuSEERV55MSmTjjDUiPYatzmb2FfQE5VS39VXfaZDpRDmWPZ0LSVESymUeyJQLEQ==->5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== -->
 
 sRevcBWen3xKQww4RWFjvUpuSEERV55MSmTjjDUiPYatzmb2FfQE5VS39VXfaZDpRDmWPZ0LSVESymUeyJQLEQ==->5jtgoAZ0ZiGVvplWjzYnvgeOrs6bbCazL7kR6KBf+FbeszWT6/MqAcS4HXP+GXMHsiiRW34gknLQl23HXLWUrw== 
 
 
 course 
 
<!-- sRevcBWen3xKQww4RWFjvUpuSEERV55MSmTjjDUiPYatzmb2FfQE5VS39VXfaZDpRDmWPZ0LSVESymUeyJQLEQ==->1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg== -->
 
 sRevcBWen3xKQww4RWFjvUpuSEERV55MSmTjjDUiPYatzmb2FfQE5VS39VXfaZDpRDmWPZ0LSVESymUeyJQLEQ==->1RqYAtzFoLkrwiQq6ttzACDhKLTrJSdWRN8gU6wyAXY4Sj2H4o0MgsYUN5Pn4VgrjVTL2uH9CWd/NC2SqGXIFg== 
 
 
 prior

In [88]:
var titlesForCourse = Given<Course>.Match((course, facts) =>
  facts.OfType<CourseTitle>()
    .Where(title => title.course == course &&
      !facts.Any<CourseTitle>(next => next.prior.Contains(title))
    )
    .Select(title => title.title)
);

var titles = await j.Query(trigonometry, titlesForCourse);
titles

[ Trig, Other ]